<a href="https://colab.research.google.com/github/sushma-snk/Sentiment-Analysis-IMDB/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [1]:
!pip install cleantext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Import required Libraries

In [3]:
import requests
import pandas as pd
import os
import re

from cleantext import clean

import string
import nltk
from nltk.stem import PorterStemmer
nltk.download('all')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [4]:
imdb_data = pd.read_csv('/gdrive/My Drive/unikaksha/mini_projects/Sentiment_Analysis/IMDB Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [7]:
# Positive and Negative review count
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
def preprocessing_text(text):
  # Convert text to lower case
  text = text.lower()

  # Remove stop words mentioned in the below list
  stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             'rt', 's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']
  STOPWORDS = set(stopwordlist)
  text = " ".join([word for word in str(text).split() if word not in STOPWORDS])

  # Dictionary of English Contractions
  contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}
  # Regular expression for finding contractions
  contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

  # Function for expanding contractions
  def expand_contractions(text_data,contractions_dict=contractions_dict):
    def replace(match):
      return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text_data)

  # Expanding Contractions in the reviews
  text = expand_contractions(text)

  # Remove words attached with numbers
  text = re.sub(r"[A-Za-z]+\d+|\d+[A-Za-z]+",'', text)

  # Define list of puctuations using built-in funtion
  english_punctuations = string.punctuation
  punctuations_list = english_punctuations
  translator = str.maketrans('', '', punctuations_list)

  # Remove punctuations
  text = text.translate(translator)

  # Remove Numbers
  text = re.sub('[0-9]+', '', text)

  # Remove unwanted (special) characters
  text = re.sub(r'[^\w]', ' ', text)

  # Remove urls
  text = re.sub('((www.[^s]+)|(https?://[^s]+))',' ',text)

  # Remove extra spaces
  text = re.sub(' +',' ',text)

  return text


In [9]:
processed_text = pd.DataFrame(imdb_data['review'].apply(preprocessing_text))

In [10]:
processed_text

,review
0,one reviewers mentioned watching oz episode yo...
1,wonderful little production br br the filming ...
2,thought wonderful way spend time hot summer we...
3,basically there is family little boy jake thin...
4,petter mattei is love time money visually stun...
...,...
49995,thought movie right good job was not creative ...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,im going disagree previous comment side maltin...


In [11]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  words = text.split()
  words = [lemmatizer.lemmatize(word,pos='v') for word in words]
  return ' '.join(words)
processed_text['review'] = processed_text['review'].apply(lemmatize_words)
processed_text

,review
0,one reviewers mention watch oz episode you wil...
1,wonderful little production br br the film tec...
2,think wonderful way spend time hot summer week...
3,basically there be family little boy jake thin...
4,petter mattei be love time money visually stun...
...,...
49995,think movie right good job be not creative ori...
49996,bad plot bad dialogue bad act idiotic direct a...
49997,catholic teach parochial elementary school nun...
49998,im go disagree previous comment side maltin on...


In [12]:
#labeling the sentient data

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
imdb_data['sentiment'] = label_encoder.fit_transform(imdb_data['sentiment'])
imdb_data['sentiment']

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [13]:
preprocessed_data = pd.concat([processed_text, imdb_data['sentiment']],axis=1)
preprocessed_data

,review,sentiment
0,one reviewers mention watch oz episode you wil...,1
1,wonderful little production br br the film tec...,1
2,think wonderful way spend time hot summer week...,1
3,basically there be family little boy jake thin...,0
4,petter mattei be love time money visually stun...,1
...,...,...
49995,think movie right good job be not creative ori...,1
49996,bad plot bad dialogue bad act idiotic direct a...,0
49997,catholic teach parochial elementary school nun...,0
49998,im go disagree previous comment side maltin on...,0


In [14]:
def token_words(text):
  word_tokens = word_tokenize(text)
  return word_tokens

In [15]:
tfidf=TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, tokenizer=token_words, use_idf=True, norm='l2', smooth_idf=True)

In [16]:
x = tfidf.fit_transform(preprocessed_data['review'])
y = preprocessed_data['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.2, shuffle=False)

In [19]:
logistic_clf = LogisticRegression(random_state=0, n_jobs=-1, verbose=3, max_iter=100)
LC = logistic_clf.fit(X_train,y_train)

y_pred_logistic_clf = LC.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    8.5s finished


In [20]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_logistic_clf))

Accuracy: 0.8944


In [21]:
from sklearn.linear_model import SGDClassifier

SGD_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
SGD = SGD_clf.fit(X_train, y_train)
y_pred_SGD = SGD.predict(X_test)

In [22]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_SGD))

Accuracy: 0.8918


In [23]:
LC_conf_mat = confusion_matrix(y_test, y_pred_logistic_clf)
print(LC_conf_mat)


[[4444  549]
 [ 507 4500]]


In [25]:
SGD_conf_mat = confusion_matrix(y_test, y_pred_SGD)
print(SGD_conf_mat)

[[4427  566]
 [ 516 4491]]
